Task 1:
Scrap all the books in the following website and save it in a csv file with the next formating ['book name','price','rate','category'] price==> float rate==>int

http://books.toscrape.com/index.html

In [2]:
import requests
import csv
from bs4 import BeautifulSoup
import pandas as pd

In [23]:
url="http://books.toscrape.com/index.html"
response=requests.get(url).text
soup=BeautifulSoup(response,"html.parser")
domain="http://books.toscrape.com/"
ratings = {'One':1, 'Two':2, 'Three':3, 'Four':4, 'Five':5}
with open("ALL_BOOKS.csv","w",newline="",encoding="utf-8") as file:
    writer=csv.writer(file)
    categories =soup.find('ul', attrs={'class':'nav nav-list'}).find_all('li')[1:]
    header=['Book_Name', 'Category', 'Rating', 'Price']
    writer.writerow(header)
    for cat in categories:
        cat_name=cat.find("a").get_text().strip()
        cat_link=domain+cat.find("a").get("href")
        response=requests.get(cat_link)
        cat_soup=BeautifulSoup(response.text,"html.parser")
        pager=cat_soup.find('ul', attrs={'class':'pager'})
        books=cat_soup.find_all("article",attrs={"class":"product_pod"})
        for book in books:
            book_name=book.find("img").get("alt")
            rating=ratings[book.find("p").get("class")[-1]]
            price=book.find("p",attrs={"class":"price_color"}).get_text()[2:]
            row=[book_name,cat_name,rating,price]
            writer.writerow(row)
        if pager:
            n_pages=int(pager.find("li",attrs={"class":"current"}).get_text().strip().split()[-1])
            for i in range(2,n_pages+1):
                link=cat_link.replace("index",f"page-{i}")
                response=requests.get(link)
                cat_soup=BeautifulSoup(response.text,"html.parser")
                books=cat_soup.find_all("article",attrs={"class":"product_pod"})
                for book in books:
                    book_name=book.find("img").get("alt")
                    rating=book.find("p").get("class")[-1]
                    price=book.find("p",attrs={"class":"price_color"}).get_text()[2:]
                    row=[book_name,cat_name,rating,price]
                    writer.writerow(row)

In [24]:
df=pd.read_csv("ALL_BOOKS.csv")

In [25]:
df

,Book_Name,Category,Rating,Price
0,It's Only the Himalayas,Travel,2,45.17
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,Travel,4,49.43
2,See America: A Celebration of Our National Par...,Travel,3,48.87
3,Vagabonding: An Uncommon Guide to the Art of L...,Travel,2,36.94
4,Under the Tuscan Sun,Travel,3,37.33
...,...,...,...,...
995,Why the Right Went Wrong: Conservatism--From G...,Politics,4,52.65
996,Equal Is Unfair: America's Misguided Fight Aga...,Politics,1,56.86
997,Amid the Chaos,Cultural,1,36.58
998,Dark Notes,Erotica,5,19.19
